In [96]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import csv

driver = webdriver.Firefox()
driver.get("https://www.autotrader.co.uk/car-search?sort=price-asc&postcode=rg20db&radius=1500&make=Porsche&model=Boxster&include-delivery-option=on&year-from=2012&exclude-writeoff-categories=on")

num_lines = sum(1 for line in open('data.csv'))

# open the file in the write mode
f = open('data.csv', 'a', newline='')

time.sleep(5)

# Accept Condition
driver.switch_to.frame(driver.find_element(By.XPATH, "//iframe[contains(@id, 'sp_message_iframe_')]"))
accept_button = driver.find_element(By.CSS_SELECTOR, "button.sp_choice_type_11")
accept_button.click()
driver.switch_to.default_content()

# for each page
while True:
    time.sleep(3)
    
    # Checkout the cars!
    carlist = driver.find_elements_by_class_name("search-page__products")[0]
    cars = carlist.find_elements(By.CLASS_NAME, "js-click-handler")

    for i in range(len(cars)):
        if num_lines > 0:
            num_lines = num_lines - 1
            continue
        
        carlist = driver.find_elements_by_class_name("search-page__products")[0]
        cars = carlist.find_elements(By.CLASS_NAME, "js-click-handler")
        car = cars[i]
        
        # create the csv writer
        writer = csv.writer(f)
        
        car.click()
        time.sleep(5)

        # Accept Condition
        iframe = driver.find_elements(By.XPATH, "//iframe[contains(@id, 'sp_message_iframe_')]")
        if iframe:
            driver.switch_to.frame(iframe[0])
            accept_button = driver.find_element(By.CSS_SELECTOR, "button.sp_choice_type_11")
            accept_button.click()
            driver.switch_to.default_content()

        data = []
        # label
        price = driver.find_element(By.XPATH, "//h2[contains(@data-testid, 'advert-price')]").text.replace('£', '').replace(',', '')
        data.append(price)
        
        # f1
        mileage = driver.find_element(By.XPATH, "//span[contains(@data-testid, 'mileage')]").text.replace(',', '').replace(' ', '').replace('miles', '')
        data.append(mileage)

        keyspeclist = driver.find_element(By.XPATH, "//ul[contains(@data-gui, 'key-specs-section')]").find_elements_by_tag_name('li')

        for spec in keyspeclist:
            if spec.find_elements_by_xpath("./p[contains(text(), 'Engine')]"):
                engine_p = spec.find_element_by_tag_name("div").find_element_by_tag_name("p")
                # f2
                engine_p = engine_p.text.replace('L', '')
                data.append(engine_p)

            if spec.find_elements_by_xpath("./p[contains(text(), 'Gearbox')]"):
                gear_p = spec.find_element_by_tag_name("div").find_element_by_tag_name("p")
                # f3
                gear_p = 1 if gear_p.text == "Automatic" else 0
                data.append(gear_p)

        readmore = driver.find_elements(By.XPATH, "//a[contains(@data-gui, 'db-read-more')]")
        if readmore:
            readmore[0].click()
            sidedoc = driver.find_element(By.XPATH, "//section[contains(@role, 'document')]").text.lower()
        else:
            driver.back()
            continue 

        # f4
        colour = -1
        if "white," in sidedoc:
            colour = 0
        elif "black," in sidedoc:
            colour = 1
        elif "grey," in sidedoc:
            colour = 2
        elif "red," in sidedoc:
            colour = 3
        elif "silver," in sidedoc:
            colour = 4
        elif "blue," in sidedoc:
            colour = 5
        elif "yellow," in sidedoc:
            colour = 5
        data.append(colour)

        # f5
        locdate = sidedoc.find("vehicle registered:")
        date = sidedoc[locdate+26:locdate+30]
        data.append(date)

        # f6
        sports_chrono = 0
        if "chrono" in sidedoc:
            sports_chrono = 1
        data.append(sports_chrono)

        # f7
        twentyin = 0
        if "20in" in sidedoc:
            twentyin = 1
        data.append(twentyin)

        # f8
        bluetooth = 0
        if "bluetooth" in sidedoc:
            bluetooth = 1
        data.append(bluetooth)

        # f9
        parkassist = 0
        if "park assist" in sidedoc:
            parkassist = 0
        data.append(parkassist)

        # f10
        heated_seat = 0
        if "heated seats" in sidedoc:
            heated_seat = 1
        data.append(heated_seat)

        # f10
        folding_mirror = 0
        if "electric folding" in sidedoc:
            folding_mirror = 1
        data.append(folding_mirror)

        # f11
        bose = 0
        if "bose" in sidedoc:
            bose = 1
        data.append(bose)

        # f12
        se = 0
        if "sports exhaust" in sidedoc:
            se = 1
        data.append(se)

        # f13
        sd = 0
        if "sport design" in sidedoc:
            sd = 1
        data.append(sd)

        # f14
        redleather = 0
        if "natural leather - carrera red" in sidedoc:
            redleather = 1
        data.append(redleather)

        # f15
        cruise_control = 0
        if "cruise control" in sidedoc:
            cruise_control = 1
        data.append(cruise_control)

        # f16
        nav = 0
        if "navigation" in sidedoc:
            nav = 1
        data.append(nav)

        # f17
        pc = 0
        if "porsche crest" in sidedoc:
            pc = 1
        data.append(pc)

        # f18
        alarm = 0
        if "alarm" in sidedoc:
            alarm = 1
        data.append(alarm)

        # f19
        wd = 0
        if "wind deflector" in sidedoc:
            wd = 1
        data.append(wd)

        # f20
        tpm = 0
        if "tpm" in sidedoc or "tyre pressure" in sidedoc:
            tpm = 1
        data.append(tpm)
        print(data)
        driver.back()
        
        # write a row to the csv file
        writer.writerow(data)
        f.flush()
        
    time.sleep(5)
    nextpage = driver.find_elements(By.XPATH, "//a[contains(@class, 'pagination--right__active')]")
    if nextpage:
        nextpage[0].click()
    else:
        break
        
# close the file
f.close()

['27995', '62400', '2.7', 1, 1, '2014', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['28199', '71362', '2.7', 0, 1, '2015', 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1]
['28280', '42000', '2.7', 0, 3, '2014', 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1]
['25995', '37000', '2.7', 0, -1, '2013', 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0]
['28495', '47000', '2.7', 1, 0, '2014', 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0]
['28495', '52650', '2.7', 1, 3, '2014', 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0]
['28495', '31000', '2.7', 0, 0, '2014', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1]
['28499', '37854', '3.4', 0, 0, '2012', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0]
['28750', '42064', '2.7', 1, 3, '2016', 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0]
['28950', '18000', '2.7', 1, 5, '2012', 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['28990', '33000', '2.7', 0, 3, '2012', 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['28990', '75000', '3.4', 1, 4,

['41990', '11900', '3.4', 1, 0, '2014', 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1]
['42900', '22136', '3.4', 1, 3, '2015', 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0]
['42995', '13000', '3.4', 1, -1, '2015', 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1]
['43999', '34284', '3.4', 1, 3, '2016', 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1]
['44950', '5267', '3.4', 1, 4, '2015', 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0]
['44995', '21000', '2.5', 1, 1, '2017', 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0]
['45750', '15579', '3.4', 1, 1, '2016', 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0]
['28199', '71362', '2.7', 0, 1, '2015', 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1]
['49090', '40436', '3.4', 1, 0, '2015', 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1]
['49500', '23388', '3.4', 1, 3, '2014', 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0]
['52990', '36208', '3.4', 1, 0, '2016', 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1]
['53650', '25710', '3.4', 1, 0, 

In [91]:
print(cars)

[<selenium.webdriver.firefox.webelement.FirefoxWebElement (session="5d90a8cc-ab32-4f12-a1d9-e4c9bbcf0902", element="f1245d6d-7223-423e-ad38-11d9415b865b")>, <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="5d90a8cc-ab32-4f12-a1d9-e4c9bbcf0902", element="6fa44d1d-a637-4d6b-bb5b-fba7bb3c2fe0")>, <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="5d90a8cc-ab32-4f12-a1d9-e4c9bbcf0902", element="4913d6fc-ae85-4c7e-bc3c-3231964e6f12")>, <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="5d90a8cc-ab32-4f12-a1d9-e4c9bbcf0902", element="ff1a8a40-4bbf-4aaf-bd5b-f81a4480f55b")>, <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="5d90a8cc-ab32-4f12-a1d9-e4c9bbcf0902", element="adfec32b-a3c2-45bd-8f2e-b969cf002313")>, <selenium.webdriver.firefox.webelement.FirefoxWebElement (session="5d90a8cc-ab32-4f12-a1d9-e4c9bbcf0902", element="b1d5331a-2415-4650-aedb-d2a5ab3e0c64")>, <selenium.webdriver.firefox.webelement.FirefoxWebElement (sessi